In [13]:
import os
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix
import gzip

In [14]:
data_path = 'D:\\Subjects_In_University\\Machine_Learning\\data\\KNN_Multinomial_Logistic_Regression'
train_images_path = os.path.join(data_path, 'train-images-idx3-ubyte.gz')
train_labels_path = os.path.join(data_path, 'train-labels-idx1-ubyte.gz')
test_images_path = os.path.join(data_path, 't10k-images-idx3-ubyte.gz')
test_labels_path = os.path.join(data_path, 't10k-labels-idx1-ubyte.gz')



In [15]:
def get_mnist_data(images_path, labels_path, num_images, shuffle=False, _is=True, image_size=28):
    f_images = gzip.open(images_path, 'r')
    f_images.read(16)
    real_num = num_images if not shuffle else (60000 if _is else 10000)
    buf_images = f_images.read(image_size * image_size * real_num)
    images = np.frombuffer(buf_images, dtype=np.uint8).astype(np.float32)
    images = images.reshape(real_num, image_size, image_size)
    
    f_labels = gzip.open(labels_path, 'r')
    f_labels.read(8)
    labels = np.zeros((real_num)).astype(np.int64)
    for i in range(0, real_num):
        buf_labels = f_labels.read(1)
        labels[i] = np.frombuffer(buf_labels, dtype=np.uint8).astype(np.int64)
    
    if shuffle:
        rand_id = np.random.randint(real_num, size=num_images)
        images = images[rand_id, :]
        labels = labels[rand_id]
    
    images = images.reshape(num_images, image_size * image_size)
    return images, labels



In [16]:
train_images, train_labels = get_mnist_data(train_images_path, train_labels_path, 5000, shuffle=True)
test_images, test_labels = get_mnist_data(test_images_path, test_labels_path, 10000, _is=False, shuffle=True)
print("Kích thước tập huấn luyện:", train_images.shape, train_labels.shape)
print("Kích thước tập kiểm tra:", test_images.shape, test_labels.shape)

train_images = train_images / 255.0  
test_images = test_images / 255.0



Kích thước tập huấn luyện: (5000, 784) (5000,)
Kích thước tập kiểm tra: (10000, 784) (10000,)


In [17]:
d = 784  
C = 10  
reg = 0.05  
lr = 0.1    
batch_size = 100
num_iters = 1000

W = np.random.randn(d, C) * 0.01 

def svm_loss_vectorized(W, X, y, reg):
    d, C = W.shape
    _, N = X.shape
    Z = W.T.dot(X) 
    correct_class_score = np.choose(y, Z).reshape(N, 1).T  
    margins = np.maximum(0, Z - correct_class_score + 1)  
    margins[y, np.arange(margins.shape[1])] = 0  
    
    loss = np.sum(margins, axis=(0, 1)) / N
    loss += 0.5 * reg * np.sum(W * W)
    
    F = (margins > 0).astype(int)
    F[y, np.arange(F.shape[1])] = np.sum(-F, axis=0)
    dW = X.dot(F.T) / N + reg * W
    return loss, dW

def multiclass_svm_GD_vectorized(X, y, Winit, reg, lr=0.1, batch_size=100, num_iters=1000, print_every=100):
    W = Winit
    loss_history = np.zeros((num_iters))
    for it in range(num_iters):
        idx = np.random.choice(X.shape[1], batch_size)
        X_batch = X[:, idx]
        y_batch = y[idx]
        loss_history[it], dW = svm_loss_vectorized(W, X_batch, y_batch, reg)
        W -= lr * dW
        if it % print_every == 1:
            print(f'it {it}/{num_iters}, loss = {loss_history[it]:.4f}')
    return W, loss_history



In [18]:
X_train = train_images.T  
y_train = train_labels    
W_opt, loss_history = multiclass_svm_GD_vectorized(X_train, y_train, W, reg, lr, batch_size, num_iters, print_every=100)

X_test = test_images.T  
y_test = test_labels   
Z_test = W_opt.T.dot(X_test) 
y_pred = np.argmax(Z_test, axis=0) 

accuracy = accuracy_score(y_test, y_pred)
print(f"Độ chính xác trên tập kiểm tra: {accuracy:.4f}")

conf_matrix = confusion_matrix(y_test, y_pred)
print("Ma trận nhầm lẫn:\n", conf_matrix)

it 1/1000, loss = 6.1755
it 101/1000, loss = 0.7758
it 201/1000, loss = 0.8348
it 301/1000, loss = 0.8619
it 401/1000, loss = 0.9526
it 501/1000, loss = 0.8438
it 601/1000, loss = 0.6020
it 701/1000, loss = 0.9166
it 801/1000, loss = 0.7235
it 901/1000, loss = 0.8784
Độ chính xác trên tập kiểm tra: 0.8761
Ma trận nhầm lẫn:
 [[ 844    0    7   32    0   30   22    0    2    0]
 [   0 1088    4   10    0    2    4    0   25    0]
 [   7    3  860   59   14    4   12   14   56    6]
 [   0    0   20  976    2   26    2    1   19   12]
 [   0    4   15    3  875    0   14    1   21  111]
 [   4    3    9   83   10  714   12    3   42   13]
 [   5    3   19    6   29    3  830    1    3    0]
 [   1   14   52   12    6    5    1  849   23   86]
 [   7    2   10   50    6   29    4    3  823   28]
 [   8    6    5   18   30    9    2    5    5  902]]
